Here, insert some new data and apply the model.

Describe whether this would be batch or online processing. Probably batch. But can still be called as a service.

It can be calculated once in the beginning of the week for the whole cohort of the clients, saved it to a database or a Google Sheet from where it can be accessed by the call center employees. (filter predicted as worthy to call to and save them as a Google Sheet)

model monitoring should be set up - maybe a week after the predictions were made considering that all the calls were made during that week and ground truth collected. a model can be evaluated and if needed, retrained with new observations.

for redeployment: model training should also be put into script and then it can be deployed to run automatically.

In [1]:
import os
import sys
import json

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import make_scorer, recall_score
from sklearn.metrics import confusion_matrix

import joblib
import pickle

In [2]:
modules_path = os.path.join(os.getcwd(), "../modules")
if modules_path not in sys.path:
    sys.path.append(modules_path)
from ml_pipeline import predict_call_worthiness

In [3]:
data_path = os.path.join(os.getcwd(), "../data/data_extracted/bank-additional")
df = pd.read_csv(os.path.join(data_path, "bank-additional.csv"), delimiter=";")

In [4]:
predictions = predict_call_worthiness(
        df,
        model_path=os.path.join(os.getcwd(), "../models/best_rf_classifier.sav"),
        threshold_path=os.path.join(os.getcwd(), "../models/best_threshold.json"),
        ohe_path=os.path.join(os.getcwd(), "../models/ohe_encoder.pkl")
    )

In [5]:
predictions

,Prediction,Probability-no,Probability-yes
0,1,0.583067,0.416933
1,0,0.786346,0.213654
2,0,0.739658,0.260342
3,0,0.771200,0.228800
4,0,0.719922,0.280078
...,...,...,...
4114,1,0.657527,0.342473
4115,0,0.748840,0.251160
4116,1,0.541988,0.458012
4117,1,0.703584,0.296416


## REST request

In [6]:
import requests
import os

In [7]:
# Define the API endpoint
url = 'http://localhost:5000/predict'  # Replace with the actual URL if deployed elsewhere

# Prepare the input data in JSON format
data_path = os.path.join(os.getcwd(), "../data/data_extracted/bank-additional")
df = pd.read_csv(os.path.join(data_path, "bank-additional.csv"), delimiter=";").head(3)
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no


In [8]:
data = df.to_json()
data = json.loads(data)

In [9]:
# Send the POST request
response = requests.post(url, json=data)

# Print the response from the server
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print(f'Error: {response.text}')

{'Prediction': {'0': 1, '1': 0, '2': 0}, 'Probability-no': {'0': 0.5830667347, '1': 0.7863463619, '2': 0.7396581773}, 'Probability-yes': {'0': 0.4169332653, '1': 0.2136536381, '2': 0.2603418227}}


In [10]:
pd.DataFrame(result)

,Prediction,Probability-no,Probability-yes
0,1,0.583067,0.416933
1,0,0.786346,0.213654
2,0,0.739658,0.260342


Make the results more clear to call center employees

In [11]:
calls_list = pd.DataFrame(result)
calls_list['Action'] = ['to call' if x == 1 else 'to skip' for x in calls_list['Prediction']]
calls_list

,Prediction,Probability-no,Probability-yes,Action
0,1,0.583067,0.416933,to call
1,0,0.786346,0.213654,to skip
2,0,0.739658,0.260342,to skip


In [ ]:
# save as a google sheet